<a href="https://colab.research.google.com/github/lorenzoram/aiayn/blob/main/aiayn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

The paper "Attention Is All You Need" introduces the Transformer model, a breakthrough in deep learning architecture for tasks involving sequences, such as translation and text generation. The key contributions are:

- Attention Mechanisms: The model relies solely on attention mechanisms, dispensing with the need for recurrence (RNNs) or convolutions (CNNs).
- Parallelization: The design allows for significant parallelization, making the model more efficient to train.
- Performance: The Transformer achieves state-of-the-art results in machine translation tasks, outperforming previous models in both speed and accuracy.

# Understanding




Main Concepts
- Self-Attention: Each word in a sequence is compared to every other word, allowing the model to focus on different parts of the input sequence when generating each word of the output.
- Multi-Head Attention: Multiple attention mechanisms run in parallel, enabling the model to capture different aspects of the input sequence simultaneously.

Methods
- Encoder-Decoder Structure:
  - Encoder: Consists of layers that apply self-attention and feed-forward neural networks to transform the input sequence into a set of continuous representations.
  - Decoder: Uses these representations to generate the output sequence, applying self-attention to the output and attention to the encoder's output.

- Positional Encoding: Adds information about the order of words in the sequence, addressing the lack of sequential information in the model's architecture.

Results

- BLEU Scores: The Transformer outperforms previous models in BLEU scores (a metric for evaluating the quality of text which has been machine-translated from one language to another), particularly in English-to-German and English-to-French translation tasks.
- Efficiency: Due to its parallelizable architecture, the Transformer trains faster and requires less computational resources compared to RNNs and CNNs.

By leveraging attention mechanisms, the Transformer model introduces a more efficient and powerful way to handle sequence transduction tasks, setting a new standard in natural language processing.

# Reproduction

## Dataset

In [2]:
import pandas as pd

First we get the dataset and preprocess it by taking only those instances that between 5 and 10 words.

In [3]:
url = 'https://raw.githubusercontent.com/lorenzoram/aiayn/main/spa.txt'

In [4]:
data = pd.read_csv(url, delimiter='\t', header=None, names=['english', 'spanish', 'metadata'])
data= data[['english','spanish']]

In [5]:
print(len(data))
data

139013


,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
...,...,...
139008,A carbon footprint is the amount of carbon dio...,Una huella de carbono es la cantidad de contam...
139009,Since there are usually multiple websites on a...,Como suele haber varias páginas web sobre cual...
139010,"If you want to sound like a native speaker, yo...","Si quieres sonar como un hablante nativo, debe..."
139011,It may be impossible to get a completely error...,Puede que sea imposible obtener un corpus comp...


In [6]:
filtered_data = data[(data['english'].str.split().apply(len) > 5) & (data['english'].str.split().apply(len) < 10)]
filtered_data = filtered_data[(filtered_data['spanish'].str.split().apply(len) > 5) & (filtered_data['spanish'].str.split().apply(len) < 10)]

print(filtered_data[0])

KeyError: 0

In [88]:
filtered_data = filtered_data.drop_duplicates(subset='english').drop_duplicates(subset='spanish')
filtered_data = filtered_data.reset_index(drop=True)

In [89]:
data= filtered_data
print(len(data))
data

38053


,english,spanish
0,I do it once a year.,Lo hago una vez al año.
1,I had to let Tom go.,Tuve que dejar que Tom se fuera.
2,I have to go get it.,Tengo que ir a por ello.
3,I have to go to bed.,Me tengo que ir a la cama.
4,Is it a yes or a no?,¿Eso es un sí o un no?
...,...,...
38048,The government watched the activities of radic...,El gobierno vigilaba atentamente las actividad...
38049,Have you recently had any unexplained dizzines...,¿Has tenido recientemente algún mareo o aturdi...
38050,"Switzerland is situated between France, Italy,...","Suiza queda entre Francia, Italia, Austria y A..."
38051,Civilization is the limitless multiplication o...,La civilización es la ilimitada multiplicación...


## Simple Convolutional Neural Network (CNN) model

In [91]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense

In [95]:
# Tokenization and padding
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 20

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(data['english'])

english_sequences = tokenizer.texts_to_sequences(data['english'])
spanish_sequences = tokenizer.texts_to_sequences(data['spanish'])

english_data = pad_sequences(english_sequences, maxlen=MAX_SEQUENCE_LENGTH)
spanish_data = pad_sequences(spanish_sequences, maxlen=MAX_SEQUENCE_LENGTH)


In [96]:
# Build CNN model
model = Sequential([
    Embedding(MAX_NUM_WORDS, 128, input_length=MAX_SEQUENCE_LENGTH),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dense(MAX_SEQUENCE_LENGTH, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [97]:
# Split data into training and validation sets
split_fraction = 0.8
num_samples = english_data.shape[0]
indices = np.arange(num_samples)
np.random.shuffle(indices)

train_samples = int(split_fraction * num_samples)

X_train = english_data[indices[:train_samples]]
X_val = english_data[indices[train_samples:]]
y_train = spanish_data[indices[:train_samples]]
y_val = spanish_data[indices[train_samples:]]


In [98]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
952/952 [==============================] - 21s 20ms/step - loss: -380500705280.0000 - accuracy: 0.5849 - val_loss: -2375005765632.0000 - val_accuracy: 0.5928
Epoch 2/10
952/952 [==============================] - 20s 21ms/step - loss: -21193486761984.0000 - accuracy: 0.5856 - val_loss: -60621101989888.0000 - val_accuracy: 0.5928
Epoch 3/10
952/952 [==============================] - 18s 19ms/step - loss: -171713648132096.0000 - accuracy: 0.5856 - val_loss: -343467930681344.0000 - val_accuracy: 0.5928
Epoch 4/10
952/952 [==============================] - 22s 23ms/step - loss: -663551710593024.0000 - accuracy: 0.5856 - val_loss: -1105036230787072.0000 - val_accuracy: 0.5928
Epoch 5/10
952/952 [==============================] - 19s 20ms/step - loss: -1775203264757760.0000 - accuracy: 0.5856 - val_loss: -2655386616201216.0000 - val_accuracy: 0.5928
Epoch 6/10
952/952 [==============================] - 20s 21ms/step - loss: -3860631211999232.0000 - accuracy: 0.5856 - val_loss: -539

In [99]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


238/238 [==============================] - 1s 3ms/step - loss: -38036853143633920.0000 - accuracy: 0.5928
Validation Accuracy: 59.28%


# Encoder

![alt text](images/aiayn/encoder.png "Encoder")

In [ ]:
import tensorflow as tf
Layer = tf.keras.layers.Layer
# This is the right layer
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)


class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super.__init__()
        self.multi_head_attention = tf.keras.layers.MultiHeadAttention(**kwargs)
        # Add & Norm
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()


class CrossAttentionLayer(AttentionLayer):
    def call(self, x, context):
        attention_output = self.multi_head_attention(query=x, value=context, key=context, return_attention_scores=True)
        x = self.add([x, attention_output])
        x = self.layer_norm(x)
        return x
        # 2nd layer

class GlobalSelfAttention(AttentionLayer):
    def call(self, x):
        attn_output = self.multi_head_attention(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layer_norm(x)
        return x

# We also modify the self-attention
# sub-layer in the decoder stack to prevent positions from attending to subsequent positions
class CausalSelfAttention(AttentionLayer):
    def call(self, x):
        attn_output = self.multi_head_attention(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
        x = self.add([x, attn_output])
        x = self.layer_norm(x)
        return x
    
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x
    
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x
  

class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.
  

class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttentionLayer(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x
  
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x
  
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits
  
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1


## Transformer model

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers

2024-06-14 14:49:10.159046: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 14:49:10.570810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 14:49:11.464645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [115]:
# Tokenization and padding
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 20

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(data['english'] + data['spanish'])

english_sequences = tokenizer.texts_to_sequences(data['english'])
spanish_sequences = tokenizer.texts_to_sequences(data['spanish'])

english_data = pad_sequences(english_sequences, maxlen=MAX_SEQUENCE_LENGTH)
spanish_data = pad_sequences(spanish_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [116]:
# Positional Encoding
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

In [117]:
# Transformer Components
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim),])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [118]:
# Parameters
vocab_size = MAX_NUM_WORDS
embed_dim = 32
num_heads = 2
ff_dim = 32
maxlen = MAX_SEQUENCE_LENGTH

In [119]:
# Build Transformer model
inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(MAX_NUM_WORDS, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [120]:
# Split data into training and validation sets
split_fraction = 0.8
num_samples = english_data.shape[0]
indices = np.arange(num_samples)
np.random.shuffle(indices)

train_samples = int(split_fraction * num_samples)

X_train = english_data[indices[:train_samples]]
X_val = english_data[indices[train_samples:]]
y_train = spanish_data[indices[:train_samples]]
y_val = spanish_data[indices[train_samples:]]


In [121]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Extension

# Conclusion